In [ ]:
import json
import bigbench.models.huggingface_models as huggingface_models
from captum.attr import Lime
from captum.attr import visualization as viz
import torch
from transformers import GPT2Tokenizer
from IPython.core.display import HTML, display

def show_text_attr_latex(tokens, attrs):
    def rgb(x): return str(128/255) + ',0,0' if x < 0 else '0,' + str(128/255) + ',0'
    token_marks = [
        (token.replace("_", "\\_").replace("%", "\\%"), attr)
        for token, attr in zip(tokens, attrs.tolist())
    ]
    token_marks = [
        f'\\textcolor[rgb]{{{rgb(attr)}}}{{{token}}}'
        for token, attr in token_marks
    ]

    print(' '.join(token_marks))


def show_text_attr_html(tokens, attrs):
    def rgb(x): return str(128/255) + ',0,0' if x < 0 else '0,' + str(128/255) + ',0'
    token_marks = [
        (token.replace("_", "\\_").replace("%", "\\%"), attr)
        for token, attr in zip(tokens, attrs.tolist())
    ]
    token_marks = [
        f'<span style="color: rgb({rgb(attr)})">{token}</span>'
        for token, attr in token_marks
    ]

    display(HTML(' '.join(token_marks)))

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions


def load_data(path):
    with open(path, 'r') as f:
        return json.load(f)
    

def predict(inputs):
    text = [tokenizer.decode(input) for input in inputs]
    return torch.tensor([model.cond_log_prob(t, "He's recovering") for t in text])

# input = """"Chapter 10: Southward Ho! \n\n\n\nUpon making inquiries, Ned Hearne found that Captain Drake had, upon the return of his expedition, set aside the shares of the prize money of Gerald Summers, himself, and the men who were lost in the wreck of the prize, in hopes that they would some day return to claim them. Upon the evidence given by Gerald and himself of the death of the others, their shares were paid, by the bankers at Plymouth who had charge of them, to their families; while Ned and Gerald received their portions. \n\nOwing to the great mortality which had taken place among the crews, each of the lads received a sum of nearly a thousand pounds, the total capture amounting to a value of over a million of money. As boys, they each received the half of a man's share. The officers, of course, had received larger shares; and the merchants who had lent money to get up the expedition gained large profits. \n\nNed thought, at first, of embarking his money in the purchase of a share in a trading vessel, and of taking to that service; but, hearing that Captain Drake intended to fit out another expedition, he decided to wait for that event, and to make one more voyage to the Spanish main, before determining on his future course. Having, therefore, his time on his hands, he accepted the invitation of the parents of his three boy friends, Tom Tressilis, Gerald Summers, and Reuben Gail. He was most warmly welcomed, for both Tom and Gerald declared that they owed their lives to him. He spent several weeks at each of their homes, and then returned to Plymouth, where he put himself into the hands of a retired master mariner, to learn navigation and other matters connected with his profession, and occupied his spare time in studying the usual branches of a gentleman's education. 
# who wanted to spend his money ?
# """

# input = """"Chapter 10: Southward Ho! \n\n\n\nUpon making inquiries, Ned Hearne found that Captain Drake had, upon the return of his expedition, set aside the shares of the prize money of Gerald Summers, himself, and the men who were lost in the wreck of the prize, in hopes that they would some day return to claim them. Upon the evidence given by Gerald and himself of the death of the others, their shares were paid, by the bankers at Plymouth who had charge of them, to their families; while Ned and Gerald received their portions. \n\nOwing to the great mortality which had taken place among the crews, each of the lads received a sum of nearly a thousand pounds, the total capture amounting to a value of over a million of money. As boys, they each received the half of a man's share. The officers, of course, had received larger shares; and the merchants who had lent money to get up the expedition gained large profits. \n\nNed thought, at first, of embarking his money in the purchase of a share in a trading vessel, and of taking to that service; but, hearing that Captain Drake intended to fit out another expedition, he decided to wait for that event, and to make one more voyage to the Spanish main, before determining on his future course. Having, therefore, his time on his hands, he accepted the invitation of the parents of his three boy friends, Tom Tressilis, Gerald Summers, and Reuben Gail. He was most warmly welcomed, for both Tom and Gerald declared that they owed their lives to him. He spent several weeks at each of their homes, and then returned to Plymouth, where he put himself into the hands of a retired master mariner, to learn navigation and other matters connected with his profession, and occupied his spare time in studying the usual branches of a gentleman's education. 
# I mean, who wants to spend money on him?
# """
# Answer: Ned


# input = """"CHAPTER XXIX \n\nFrona had gone at once to her father's side, but he was already recovering. Courbertin was brought forward with a scratched face, sprained wrist, and an insubordinate tongue. To prevent discussion and to save time, Bill Brown claimed the floor. \n\n\"Mr. Chairman, while we condemn the attempt on the part of Jacob Welse, Frona Welse, and Baron Courbertin to rescue the prisoner and thwart justice, we cannot, under the circumstances, but sympathize with them. There is no need that I should go further into this matter. You all know, and doubtless, under a like situation, would have done the same. And so, in order that we may expeditiously finish the business, I make a motion to disarm the three prisoners and let them go.\" \n\nThe motion was carried, and the two men searched for weapons. Frona was saved this by giving her word that she was no longer armed. The meeting then resolved itself into a hanging committee, and began to file out of the cabin. \n\n\"Sorry I had to do it,\" the chairman said, half-apologetically, half-defiantly. \n\nJacob Welse smiled. \"You took your chance,\" he answered, \"and I can't blame you. I only wish I'd got you, though.\" \n\nExcited voices arose from across the cabin. \"Here, you! Leggo!\" \"Step on his fingers, Tim!\" \"Break that grip!\" \"Ouch! Ow!\" \"Pry his mouth open!\" \n\nFrona saw a knot of struggling men about St. Vincent, and ran over. He had thrown himself down on the floor and, tooth and nail, was fighting like a madman. Tim Dugan, a stalwart Celt, had come to close quarters with him, and St. Vincent's teeth were sunk in the man's arm. 
# How is Frona's father doing?
# """

input = """"CHAPTER XXIX \n\nFrona had gone at once to her father's side, but he was already recovering. Courbertin was brought forward with a scratched face, sprained wrist, and an insubordinate tongue. To prevent discussion and to save time, Bill Brown claimed the floor. \n\n\"Mr. Chairman, while we condemn the attempt on the part of Jacob Welse, Frona Welse, and Baron Courbertin to rescue the prisoner and thwart justice, we cannot, under the circumstances, but sympathize with them. There is no need that I should go further into this matter. You all know, and doubtless, under a like situation, would have done the same. And so, in order that we may expeditiously finish the business, I make a motion to disarm the three prisoners and let them go.\" \n\nThe motion was carried, and the two men searched for weapons. Frona was saved this by giving her word that she was no longer armed. The meeting then resolved itself into a hanging committee, and began to file out of the cabin. \n\n\"Sorry I had to do it,\" the chairman said, half-apologetically, half-defiantly. \n\nJacob Welse smiled. \"You took your chance,\" he answered, \"and I can't blame you. I only wish I'd got you, though.\" \n\nExcited voices arose from across the cabin. \"Here, you! Leggo!\" \"Step on his fingers, Tim!\" \"Break that grip!\" \"Ouch! Ow!\" \"Pry his mouth open!\" \n\nFrona saw a knot of struggling men about St. Vincent, and ran over. He had thrown himself down on the floor and, tooth and nail, was fighting like a madman. Tim Dugan, a stalwart Celt, had come to close quarters with him, and St. Vincent's teeth were sunk in the man's arm. 
What's Frona's father doing?
"""
# Answer: He's recovering 

model = huggingface_models.BIGBenchHFModel('gpt2-large')

# Tokenize the input
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
input_ids = tokenizer.encode(input, return_tensors='pt')

lime = Lime(predict)

explanation = lime.attribute(input_ids.squeeze(), target=0, show_progress=True)

# Decode the entire sequence of token IDs into a string
text = tokenizer.decode(input_ids.squeeze())

# Split the string into individual tokens
tokens = text.split()

# Call show_text_attr with the list of tokens and the attributions
show_text_attr_html(tokens, explanation)
